In [1]:
import pm4py
pm4py.__version__

'1.2.12'

# Social Network Analysis

> Discover the organizational perspective of the process. For each of the following networks, try to find a clear organizational structure and discuss the structure obtained. If no clear structure is to be found, explain why this is the case.

## Data loading

Import the log.

In [2]:
import os
import pandas as pd
from pm4py.objects.conversion.log import factory as log_converter
from pm4py.util import constants

PROJ_ROOT = os.path.abspath(os.path.pardir)
data_path = os.path.join(PROJ_ROOT, 'data', 'log.csv')

assert os.path.exists(data_path)

df_log = pd.read_csv(data_path)

# map dataset columns to PM4Py keys
param_keys={constants.PARAMETER_CONSTANT_CASEID_KEY: 'Patient',
            constants.PARAMETER_CONSTANT_RESOURCE_KEY: 'Resource', 
            constants.PARAMETER_CONSTANT_ACTIVITY_KEY: 'Activity',
            constants.PARAMETER_CONSTANT_TIMESTAMP_KEY: 'Timestamp',
            constants.PARAMETER_CONSTANT_START_TIMESTAMP_KEY: 'start_timestamp'}

event_log = log_converter.apply(df_log, parameters=param_keys)

---

# a)
> Handover-of-Work Social Network

In [3]:
# define the path where the resulting networks will be stored
RESULT_PATH = os.path.join(PROJ_ROOT, 'results/Q2/')

In [4]:
from pm4py.algo.enhancement.sna import factory as sna_factory
from pm4py.visualization.sna import factory as sna_vis_factory

#discover the network
handover_net = sna_factory.apply(event_log, parameters=param_keys, variant="handover")

#create a visualization of the network
handover_viz = sna_vis_factory.apply(handover_net, variant="pyvis")

#save the visualization of the discovered network
sna_vis_factory.save(handover_viz, RESULT_PATH+"handover_net.html", variant="pyvis")


As the pyvis interactive visualization cannot be embedded, the discovered network is shown below in the form of an image. The real network can be found in the results/Q2 folder.

![](../report/figures/q2_handover.png)

A handover-of-work network connects two resources when an activity performed by one resource directly follows the activity performed by another resource. The more frequent this relationship is in the log, the higher is the weight of an edge. In the graphic above the handover-of-work network that was discovered from the given log can be seen.

We see that the network consists of four strongly connected main components and a single resource that has only one connection. Further we see that the resources in each of the strongly connected components all have a common first letter. The outer three components contain resources with first letters B, C and D. The inner component consists of resources with the first letter A. The nodes in the three outer components are connected only to themselves and the inner A component, but not to the other outer components. Most of the nodes in the A component have connections to all other components and the nodes in the A component have overall the most connections to other nodes. There is a single resource that does not belong to any of the components and has only one incoming edge.

From an orgainzational perspective the observed network might be explained in the following way: The outer components B, C, D all represent certain distinct departments. All of the resources in such a department do similar work and are working together on a certain part of the process. The central A component however consist of resources that are delegating and inspecting work. They pass process instances from one department to another. Since the singular resource has only incoming edges, it must be concerned with the end of the process. It might be a kind of supervisor role that makes a final assertion of a process before it ends. However this node is not involved in most cases, as its connection is rather weak.

Let's check the assumption of the relation between groups and activities.

In [15]:
df_log['Resource']

0            Alexander
1                 Anna
2          Amelie,Anna
3            Alexander
4                 Anna
             ...      
33859    InspectorInis
33860    InspectorInis
33861           Amalia
33862    Amelie,Amalia
33863        Alexander
Name: Resource, Length: 33864, dtype: object

In [27]:
first_person_group = df_log['Resource'].str[0]
second_person_group = df_log['Resource'].str.split(',').apply(lambda a: a[-1]).str[0]

(first_person_group != second_person_group).sum()

0

This means that when two resources share the activity, it is contained in one of the groups of strongly connected resources.

In [28]:
df_log['Component'] = first_person_group

df_log['Component'].value_counts()

I    9954
A    9147
B    5401
D    4755
C    4606
H       1
Name: Component, dtype: int64

In [34]:
for c in df_log['Component'].unique():
    print('==== Component ', c)
    print(df_log[df_log['Component'] == c]['Activity'].value_counts())

==== Component  A
Register                        1438
Initial Exam                    1436
Initial Exam Decision           1419
Test III                         779
Inform about Isolation           758
Test III Decision                745
Discharge Init Exam              687
Inform Authority Send Form       533
Inform Authority Fill Form       529
Referral                         529
Discharge Test                   216
Prescripe Special Medication      78
Name: Activity, dtype: int64
==== Component  B
Check Treatment A1    1053
Treatment A1          1047
Treatment B            738
Check Treatment B      737
Check Treatment A2     411
Treatment A2           408
Treatment A3           215
Check Treatment A3     213
Decide Treatment       194
Register Facility      193
Discharge              192
Name: Activity, dtype: int64
==== Component  D
Check Treatment A1    824
Treatment A1          811
Treatment B           795
Check Treatment B     794
Treatment A2          318
Check Treatment A

We verify that, indeed, the components represent groups of resources that act in different activities. We can estimate that groups B, C and D are responsible for the treatment of confirmed cases; group I is responsible for the control call only; H is the helicopter, only used in emergencies; and group A deals with the remaining activities related to the trial of the patients.

---

# b)
> Subcontracting Social Network

In [18]:
#discover the network
subcontracting_net = sna_factory.apply(event_log, parameters=param_keys, variant="subcontracting")

#create a visualization of the network
subcontracting_viz = sna_vis_factory.apply(subcontracting_net, variant="pyvis")

#save the visualization of the discovered network
sna_vis_factory.save(subcontracting_viz, RESULT_PATH+"subcontracting_net.html", variant="pyvis")

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecatedWarning: apply is deprecated as of 1.3.0 and will be removed in 2.0.0. Use algorithm entrypoint instead
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecatedWarning: apply is deprecated as of 1.3.0 and will be removed in 2.0.0. Use visualizer module instead.
  """
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecatedWarning: save is deprecated as of 1.3.0 and will be removed in 2.0.0. Use visualizer module instead.
  


As the pyvis interactive visualization cannot be embedded, the discovered network is shown below in the form of an image. The real network can be found in the results/Q2 folder.

![](../report/figures/q2_subcontracting.png)

A subcontracting social network visualizes what resources are involved in the work of another resource.

The obtained network that can be seen above has a kind of forest structure. We again see distinct connected components that show the same A, B, C, D division as in the handover-of-work network discussed above. There is one component for each of these, except for the A component, which now consists of two not connected components. As a common structure we can observe a kind of star pattern for many nodes, meaning that one resource is involved in the work of many other resources. We can see that one of the A components has a very strong relation between two of the resources.

From an organizational perspective we can again see that the work of one department never involves the work of another department. Subcontracting only happens within the A, B, C, D departments. The star structures suggest that there are supervisor roles, resources that often subcontract work to other resources which do not subcontract any work. The strong connection between the _Alexander_ and _Ava_ resources suggests that Alexander is in a leadership position and Ava might be her secretary, as most of the work is subcontracted.

---

# c)
> Working-Together Social Network

In [19]:
#discover the network
working_together_net = sna_factory.apply(event_log, parameters=param_keys, variant="working_together")

#create a visualization of the network
working_together_viz = sna_vis_factory.apply(working_together_net, variant="pyvis")

#save the visualization of the discovered network
sna_vis_factory.save(working_together_viz, RESULT_PATH+"working_together_net.html", variant="pyvis")

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecatedWarning: apply is deprecated as of 1.3.0 and will be removed in 2.0.0. Use algorithm entrypoint instead
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecatedWarning: apply is deprecated as of 1.3.0 and will be removed in 2.0.0. Use visualizer module instead.
  """
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecatedWarning: save is deprecated as of 1.3.0 and will be removed in 2.0.0. Use visualizer module instead.
  


As the pyvis interactive visualization cannot be embedded, the discovered network is shown below in the form of an image. The real network can be found in the results/Q2 folder.

![](../report/figures/q2_working_together.png)

The working-together network depicts which resources are often working together on the samee case, so in which fraction of cases they were both involved.

We see a similar structure as in the hondover-of-work network, which is expected. We again have four main components, with three outer ones and an inner one, following the recurring A, B, C, D pattern. We see that the connections in this network are even stronger than in the handover of work network. All of the components are almost fully connected. The inner A component is most strongly connected. Again the outer components have no connections to the other outer connections, but are all connected to the inner component. The singular resource is again present and is only connected to resources in the A component.

From an organizational point of view, we see that the resources in the A component are the most important ones. They are involved in almost all cases and are almost always working together. Again it seems that they delegate work to the outer components. The almost full connection of the components suggests, that when a department is involved in a case at all, usually the whole department is involved and not only certain resources. From the subcontracting network we have seen that in the outer components, much of the work is subcontracted, which explains the involvement of many different resources of each component we can see in the working-together network.

--- 

# d)
> Joint-Activity Network

In [20]:
#discover the network
jointactivities_net = sna_factory.apply(event_log, parameters=param_keys, variant="jointactivities")

#create a visualization of the network
jointactivities_viz = sna_vis_factory.apply(jointactivities_net, variant="pyvis")

#save the visualization of the discovered network
sna_vis_factory.save(jointactivities_viz, RESULT_PATH+"jointactivities_net.html", variant="pyvis")

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecatedWarning: apply is deprecated as of 1.3.0 and will be removed in 2.0.0. Use algorithm entrypoint instead
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecatedWarning: apply is deprecated as of 1.3.0 and will be removed in 2.0.0. Use visualizer module instead.
  """
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecatedWarning: save is deprecated as of 1.3.0 and will be removed in 2.0.0. Use visualizer module instead.
  


As the pyvis interactive visualization cannot be embedded, the discovered network is shown below in the form of an image. The real network can be found in the results/Q2 folder.

![](../report/figures/q2_jointactivities.png)

The jointactivities network depicts how similar the types of activities are that the resources perform. Meaning that two resources have a strong connection if the always perform the same e.g. three activities but never any other.

The structure of the obtained network is very different from all networks we have considered up to this point. It consists of six strongly connected components. It seems that all of the components are fully connected, meaning that all resources in a component perform very similar activities. We see that the destinction between components is no longer based on the first letter of the resource name, as we have seen before. The components are now mixtures of these resources. This is not true for one of the components, which only contains A resources.

For the organizational structure this again suggests the supervisor role for the A resources, there are certain activities that are only performed by these resources. Other activities however seem to occur in all of the B, C, D departments. Combining these findings with the findings from the other networks we cann assume that the process is structured in such a way, that the A department is a supervision department that performs certain specialized taks. The other departments B, C, D, however perform similar activities (as the jointactivities network suggest) but they never mix. This suggests that a case is delegate by the A resources to one of the departments and this department then fully handles the case. It is in a way not a delegation of cases based on certain types of cases, but just a balancing of cases over all departments.